In [ ]:
import openpyxl
import numpy as np
import pandas as pd

from tensorflow import keras
from tensorflow.keras.regularizers import l2
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dense, Dropout, LeakyReLU

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

Load & split the dataset

In [ ]:
df = pd.read_excel("../1280x321_features_binaryfocus.xlsx")

X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
import numpy as np
unique, counts = np.unique(y_train, return_counts=True)
print(dict(zip(unique, counts)))  # Shows counts of 0s vs 1s

In [ ]:
df

Define the MLP(Multilayer Perceptron) architecture

In [ ]:
model = Sequential([
    Dense(128, activation='tanh', kernel_regularizer=l2(0.01), input_shape=(X_train.shape[1],)), # input + 1st hidden layer
    Dropout(0.2), # prevent overfiting

    Dense(64, activation='tanh', kernel_regularizer=l2(0.01)), # 2nd hidden layer
    Dropout(0.2),

    Dense(32, activation='tanh', kernel_regularizer=l2(0.01)), # 3rd hidden layer
    Dropout(0.2),

    Dense(1, activation='sigmoid') # Output layer
])

Compile the model

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

Train the model

In [ ]:
early_stopping = EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True)

model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2, callbacks=[early_stopping], verbose=1)

Evaluate the model

In [ ]:
y_pred = (model.predict(X_test) > 0.5).astype(int)

print("\nAccuracy = ", accuracy_score(y_test, y_pred))
print("\nClassification report: \n\n", classification_report(y_test, y_pred))